In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [2]:
from lightning import Trainer
from torchvision.transforms import v2 as T # type: ignore
from geovision.logging import get_logger
from geovision.config.basemodels import ExperimentConfig # noqa
from geovision.data.module import ImageDatasetDataModule
from geovision.training.module import ClassificationModule
from geovision.training.loggers import get_csv_logger, get_wandb_logger, get_ckpt_logger, get_classification_logger

In [3]:
logger = get_logger("experiment_logger")
transforms: dict[str, T.Transform | None] = {
    "image_transform": None,
    "target_transform": None,
    "common_transform": None,
}
config = ExperimentConfig.from_yaml("config.yaml", transforms)
datamodule = ImageDatasetDataModule(config)

In [6]:
litmodule = ClassificationModule(ExperimentConfig.from_yaml("config.yaml", transforms))

loggers: list = list()
callbacks: list = list()
callbacks.append(get_classification_logger(config))
loggers.append(csv_logger := get_csv_logger(config))
# loggers.append(wandb_logger := get_wandb_logger(config))
callbacks.append(ckpt_logger := get_ckpt_logger(config))

# wandb_logger.watch(litmodule)
trainer = Trainer(
    logger = loggers,
    callbacks = callbacks,
    max_epochs = 1,
    check_val_every_n_epoch = 1,
    num_sanity_val_steps=0,
    #limit_val_batches=1
)

trainer.fit(
    model = litmodule,
    datamodule = datamodule
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /home/sambhav/experiments/imagenette_classification/can_imagenette_work_with_grace exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/sambhav/experiments/imagenette_classification/can_imagenette_work_with_grace/ckpts exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type                      | Params
--------------------------------------------------------------------
0 | model                 | AlexNet                   | 57.0 M
1 | criterion             | CrossEntro

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

batch #0
batch #1
batch #2
batch #3
batch #4
batch #5
batch #6
batch #7
batch #8
batch #9
batch #10
batch #11
batch #12
batch #13
batch #14
batch #15
batch #16
batch #17
batch #18
batch #19
batch #20
output file /home/sambhav/experiments/imagenette_classification/can_imagenette_work_with_grace/val_logs/epoch=0_step=147_logits.h5, final idx 1339
tensor([[129,   1,   0,   1,   0,   2,   0,   0,   0,   2],
        [  2, 124,   0,   2,   2,   1,   0,   0,   1,   3],
        [  1,   0, 119,   1,   0,   2,   2,   8,   0,   2],
        [ 10,   2,   2,  86,   2,   2,   9,   6,   0,   5],
        [  1,   0,   2,   1, 121,   3,   3,   2,   0,   2],
        [  6,   1,   4,   1,   2, 116,   2,   2,   0,   1],
        [  0,   1,   4,   4,   2,   0, 117,   7,   0,   0],
        [  0,   1,  11,   6,   6,   1,   9, 101,   0,   0],
        [  5,   0,   1,   1,   2,   3,   2,   3, 116,   2],
        [  1,   2,   0,   6,   3,   0,   2,   2,   0, 119]], device='cuda:0')


`Trainer.fit` stopped: `max_epochs=1` reached.


In [1]:
from pathlib import Path
import h5py
import torch
import torchmetrics
import numpy as np
import pandas as pd


logs_dir = Path.home()/"experiments"/"imagenette_classification"/'_'.join(config.name.split(' '))/"val_logs"
logits_file = sorted(logs_dir.rglob("*.h5"))[-1]
display(logits_file)
with h5py.File(logits_file) as f:
    logits = torch.tensor(f["logits"][:])
    labels = torch.tensor(f["labels"][:])
    df_idxs = f["df_idxs"][:]

# torchmetrics.functional.confusion_matrix(logits, labels, "multiclass", num_classes=10)
# pd.Series(logits.argmax(1).tolist()).value_counts()
def get_confusion_matrix(logits, labels):
    num_classes = logits.shape[1] 
    confm = np.zeros((num_classes, num_classes), dtype = np.uint64)
    for i in range(len(logits)):
        pred_idx = np.argmax(logits[i])
        true_idx = labels[i]
        confm[true_idx, pred_idx] += 1
    display(confm)

get_confusion_matrix(logits, labels)

NameError: name 'config' is not defined

In [17]:
# multilabel classification output
logits = torch.randn(5, dtype = torch.float32)
probs = torch.sigmoid(logits)
truth = torch.tensor([0, 1, 0, 1, 0], dtype = torch.float32)

print(probs, probs.sum())
print(truth, truth.sum())

torch.nn.BCEWithLogitsLoss()(truth, probs)

tensor([0.6044, 0.8164, 0.6411, 0.3940, 0.4034]) tensor(2.8593)


tensor(0.6991)